In [ ]:
install.packages("cvAUC") # install cvAUC package if not installed!


In [ ]:
# load relevant libraries
library(stringr)
library(cvAUC)

In [ ]:
folders <- list.dirs(path=file.path(dirname(getwd()), 'decoded_output'), full.names=T)
for (folder in folders){
#     print(folder)
    df = data.frame()
    for (file in list.files(path=folder, pattern="*_upd.txt", full.names=T, recursive=FALSE)){
        train <- read.table(file, header=1,sep="\t")
        train<- train[train[,'lastevent']==1,]
        df <- rbind(df,train)
    }
    if(nrow(df)){
        print(basename(folder))
        rownames(df) <- 1:nrow(df)
        # build folds_lst
        folds_lst = list()
        for (fold_i in 0:4){
            folds_lst[fold_i+1] <- list(as.numeric(rownames(df[df[,'fold_id'] == stringr::str_interp("fold_${fold_i}_none", list(fold_i = fold_i)), ])))
#             print(length(folds_lst[[fold_i+1]]))
        }
        print("total records: ")
        predictions <- df["prob_target1"]
        colnames(predictions)<-''
        predictions_unlst <- unlist(predictions)
        print(length(predictions_unlst))
        labels = df["ref_target"]
        colnames(labels)<-''
        labels_unlst <- unlist(labels)
        if(length(labels_unlst) != length(predictions_unlst)){
            print("Num records mismatch !!!")
        }
        
        res <- ci.cvAUC(predictions = predictions_unlst, 
                        labels = labels_unlst,
                        folds = folds_lst, 
                        confidence = 0.95)   
        cat("res: \n")
        print(res)
        cat("-------------------------- \n")
    }                                    
}
